# Ambassador Examples

Assumes

 * You have installed emissary as per their docs

In [ ]:
INGRESS_IP=!kubectl get svc emissary-ingress -n emissary -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
INGRESS_IP=INGRESS_IP[0]
import os
os.environ['INGRESS_IP'] = INGRESS_IP
INGRESS_IP

## Ambassador Single Model Example

In [ ]:
!kustomize build config/single-model

In [ ]:
!kustomize build config/single-model | kubectl apply --validate=false -f -

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!curl -v http://${INGRESS_IP}/v2/models/iris/infer -H "Content-Type: application/json"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

In [ ]:
!grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
    -plaintext \
    -import-path ../../apis \
    -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
    ${INGRESS_IP}:80 inference.GRPCInferenceService/ModelInfer

In [ ]:
!kustomize build config/single-model | kubectl delete -f -

## Traffic Split Two Models

In [ ]:
!kustomize build config/traffic-split

In [ ]:
!kustomize build config/traffic-split | kubectl apply -f -

In [ ]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

In [ ]:
!curl -v http://${INGRESS_IP}/v2/models/iris/infer -H "Content-Type: application/json" \
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

### TODO model_name needs fixing

In [ ]:
!grpcurl -d '{"model_name":"iris1","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
    -plaintext \
    -import-path ../../apis \
    -proto ../../apis/mlops/v2_dataplane/v2_dataplane.proto \
    ${INGRESS_IP}:80 inference.GRPCInferenceService/ModelInfer

In [ ]:
!kustomize build config/traffic-split | kubectl delete -f -